# Import all required packages

In [ ]:
!pip install pandas
!pip install librosa
!pip install numpy
!pip install pickle
!pip install scikit-learn
!pip install xgboost
!pip install LGBMClassifier

In [6]:
import os
import re
from tqdm import tqdm
import pandas as pd
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
from lightgbm import LGBMClassifier
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
import pickle

# Data preparation

## Reading a markup file and audio recordings

In [7]:
df = pd.read_excel('/content/mga_dialect_audio.xlsx')

In [8]:
df_data = df[['path', 'dialect']]

In [9]:
def load_audio(df_data):
    audios = []
    labels = []
    for index, row in tqdm(df_data.iterrows()):
        audio, sample_rate = librosa.load(row['path'], sr=16000)
        mfccs = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=40)
        mfccs_s = np.mean(mfccs.T, axis=0)
        audios.append(mfccs_s)
        labels.append(row['dialect'])
    return audios, labels

In [10]:
audios, labels = load_audio(df_data)
df_audio = pd.DataFrame(audios)
df_audio['label'] = labels

1823it [01:03, 28.85it/s]


# Trainig

## Split on test and train

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df_audio.drop(['label'], axis=1), df_audio['label'], test_size=0.3, random_state=22)

## Training of the model LGBMClassifier

In [42]:
clf = LGBMClassifier(max_depth=100, learning_rate=0.01, n_estimators=1000, n_jobs=-1, random_state=22, reg_alpha=0.2)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(f1_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))

0.48066298342541436
0.5471698113207547
0.42857142857142855


In [54]:
with open('clf.pkl', 'wb') as f:
    pickle.dump(clf, f)

## Training of the model XGBClassifier

In [44]:
xgb_cl = xgb.XGBClassifier(max_depth=5, learning_rate=0.01, n_estimators=1000)
xgb_cl.fit(X_train, y_train)
y_pred = xgb_cl.predict(X_test)
print(f1_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))

0.48913043478260865
0.5454545454545454
0.4433497536945813


In [55]:
with open('xgb_cl.pkl', 'wb') as f:
    pickle.dump(xgb_cl, f)

## Training of the model GaussianNB

In [45]:
gaus = GaussianNB()
gaus.fit(X_train, y_train)
y_pred = gaus.predict(X_test)
print(f1_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))

0.551440329218107
0.4734982332155477
0.6600985221674877


In [56]:
with open('gaus.pkl', 'wb') as f:
    pickle.dump(gaus, f)